**Predicting House Prices with a Feedforward Neural Network**

*Importing the dataset*: California Housing dataset available in the library sklearn

In [6]:
# pip install scikit-learn
# pip install torch torchvision torchaudio
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim

# Loading the dataset
data = fetch_california_housing()
X, y = data.data, data.target